In [ ]:
import cx_Oracle as oci
from time import sleep
import requests
from bs4 import BeautifulSoup
from multiprocessing import Process, current_process
import os

In [ ]:
url = "http://172.30.1.18:5000/post"

In [ ]:
conn = oci.connect("LOGISHR_USER", "LOGISHR_USER123",
                   "logishr.ccnh4snd35bm.ap-northeast-2.rds.amazonaws.com:1521/logishr")

In [ ]:
def select_recommend_req():
    cur = conn.cursor()
    proc = os.getpid()
    proc_name = current_process().name
    # print(proc_name)
    sql = "SELECT FN_RECOMMEND_REQ " \
          "  FROM DUAL"
    rows = ""
    while True:
        try:
            cur.execute(sql)
            rows = cur.fetchone()
            print("---입력 대기중---" + proc_name)
            if rows[0] is not None:
                print(rows[0])
                print(proc_name)
                row = rows[0].split('-')
                if row[1] == 'C':
                    select_contents(row)
                elif row[1] == 'U':
                    select_id_fav(row)
                elif row[1] == 'T':
                    select_board(row)
        except Exception as e:
            print(e)
        sleep(5)

In [ ]:
def select_contents(row):
    cur = conn.cursor()
    sql = "SELECT TITLE || ' ' || CAT1 || ' ' || CAT2 || ' ' || CAT3 || ' ' || CAT4 || ' ' || CAT5 || ' ' || CAT6 " \
          " || ' ' || CAT7 || ' ' || CAT8 || ' ' || CAT9 || ' ' || CAT10 AS CONTENTS_INFO" \
          " FROM ( " \
          " SELECT CONTENTS_ID , TITLE , CAT1 , CAT2 , CAT3 , CAT4 , CAT5 , CAT6 " \
          "     , CAT7 , CAT8 , CAT9 , CAT10 " \
          "   FROM TBH_CONTENTS_EDU " \
          "  WHERE OPEN_GUBUN = 'Y' " \
          " UNION ALL " \
          " SELECT CONTENTS_ID, TITLE , CAT1 , CAT2 , CAT3 , CAT4 , CAT5 " \
          "     , CAT6 , CAT7 , CAT8 , CAT9 , CAT10 " \
          "   FROM TBH_CONTENTS_HR " \
          "  WHERE OPEN_GUBUN = 'Y' " \
          " UNION ALL " \
          " SELECT CONTENTS_ID, TITLE , CAT1 , CAT2 , CAT3 , CAT4 , CAT5 " \
          "     , CAT6 , CAT7 , CAT8 , CAT9 , CAT10 " \
          "   FROM TBH_CONTENTS_NEWS " \
          "  WHERE OPEN_GUBUN = 'Y'" \
          " ) WHERE CONTENTS_ID=:CONTENTS_ID"
    rows = ""
    try:
        cur.execute(sql, CONTENTS_ID=row[2])
        rows = cur.fetchone()
        if rows is not None:
            result = rows[0]
            send_post(result, row, 'C')
        else:
            print("저장데이터가 없습니다.")
    except Exception as e:
        print("CONTENTS select 오류")
        print(e)
    # send_post(result, row, 'C')

In [ ]:
def select_board(row):
    cur = conn.cursor()
    sql_check_title = "SELECT A.BOARD_TITLE " \
                      " FROM TBH_BOARD A" \
                      " WHERE A.BOARD_KEY =: BOARD_KEY"
    rows_check_title = ""
    result_check_title = ""
    try:
        cur.execute(sql_check_title, BOARD_KEY=row[2])
        rows_check_title = cur.fetchone()
        result_check_title = rows_check_title[0]
    except Exception as e:
        print("BOARD TITLE SELECT 오류")
        print(e)
    # print(result_check_title)
    if len(result_check_title) < 16:
        print("제목 4자 이하")
    else:
        sql = "SELECT A.BOARD_TITLE || ' ' || A.BOARD_CONTENTS_TEXT || ' ' || B.CODE_DESC || ' ' ||" \
              " (SELECT Z.CODE_DESC FROM TBH_CODE Z WHERE Z.MINOR = A.BOARD_JOB2 AND B.VALUE1 = Z.MAJOR) AS BOARD_INFO" \
              " FROM TBH_BOARD A" \
              "    , (SELECT * FROM TBH_CODE WHERE MAJOR = 'H01') B" \
              " WHERE A.BOARD_KEY=:BOARD_KEY" \
              "   AND A.BOARD_JOB1 = B.MINOR(+)"
        rows = ""
        try:
            cur.execute(sql, BOARD_KEY=row[2])
            rows = cur.fetchone()
            result = rows[0]
        except Exception as e:
            print("BOARD select 오류")
            print(e)
        send_post(result, row, 'T')

In [ ]:
def select_id_fav(row):
    cur = conn.cursor()
    sql = "SELECT LISTAGG(INFO, ' ') WITHIN GROUP(ORDER BY INFO) AS INFO" \
          " FROM(" \
          " SELECT LISTAGG(SKILL_NAM || ' ' || SKILL_DESC, ' ') WITHIN GROUP(ORDER BY SKILL_NAM) AS INFO" \
          " FROM TBH_ID_SKILLS" \
          " WHERE ID_COD =:ID_COD" \
          " AND SKILL_NAM IS NOT NULL" \
          " UNION ALL" \
          " SELECT LISTAGG(INFO_JOB1 || ' ' || INFO_JOB2 || ' ' || INFO_COMPANY || ' ' || INTEREST_DESC) WITHIN GROUP(ORDER BY INFO_JOB1) AS INFO" \
          " FROM TBH_ID_INFO" \
          " WHERE ID_COD =:ID_COD" \
          " UNION ALL" \
          " SELECT LISTAGG(MAJOR_DESC1 || ' ' || MAJOR_DESC2 || ' ' || MAJOR_DESC3 || ' ' || MAJOR_DESC4 || ' ' || MAJOR_DESC5) WITHIN GROUP(ORDER BY MAJOR_DESC1) AS INFO" \
          " FROM TBH_ID_SCHOOL" \
          " WHERE ID_COD =:ID_COD" \
          " UNION ALL" \
          " SELECT LISTAGG(A.COMPANY_NAM || ' ' || " \
          " B.CODE_DESC || ' ' || " \
          " (SELECT Z.CODE_DESC FROM TBH_CODE Z WHERE Z.MINOR = A.JOB2_COD AND B.VALUE1 = Z.MAJOR) || ' ' || " \
          " DPT_NAM || ' ' || PRJ_DESC || ' ' || A.IMP_DESC1 || ' ' || A.IMP_DESC2 || ' ' || A.IMP_DESC3 || ' ' || A.IMP_DESC4 || ' ' || A.IMP_DESC5) " \
          " WITHIN GROUP(ORDER BY A.COMPANY_NAM) AS INFO" \
          " FROM TBH_ID_INFO_CAREER A" \
          " , (SELECT * FROM TBH_CODE WHERE MAJOR = 'H01') B" \
          " WHERE A.JOB1_COD = B.MINOR " \
          " AND ID_COD =:ID_COD" \
          " UNION ALL" \
          " SELECT LISTAGG((B.CODE_DESC) || ' ' ||" \
          " (SELECT Z.CODE_DESC FROM TBH_CODE Z WHERE Z.MINOR = A.FAV_JOB2_COD AND B.VALUE1 = Z.MAJOR)) WITHIN GROUP(ORDER BY A.FAV_JOB1_COD) AS INFO" \
          " FROM TBH_ID_FAV_JOB A" \
          " , (SELECT * FROM TBH_CODE WHERE MAJOR = 'H01') B" \
          " WHERE A.FAV_JOB1_COD = B.MINOR " \
          " AND ID_COD =:ID_COD)"

    rows = ""
    try:
        cur.execute(sql, ID_COD=row[2])
        rows = cur.fetchone()
        result = rows[0]
    except Exception as e:
        print("USER select 오류")
        print(e)

    send_post(result, row, 'U')

In [ ]:
def send_post(result, row, gubun):
    r = requests.post(url, data=result.encode('UTF-8'))
    if gubun == 'U':
        save_id_fav(r.text, row)
    elif gubun == 'T':
        save_board(r.text, row)
    elif gubun == 'C':
        save_contents(r.text, row)

In [ ]:
def save_contents(rec_text, row):
    cur = conn.cursor()
    result_ori = rec_text.split('||')[0]
    result_rev = rec_text.split('||')[1]

    result_ori2 = result_ori.split(',')
    result_rev2 = result_rev.split(',')

    result_ori_count = len(result_ori2)
    result_rev_count = len(result_rev2)

    # sql = "DELETE FROM TBH_CONTENTS_REC WHERE CONTENTS_ID =:CONTENTS_ID"
    # try:
    #     cur.execute(sql, CONTENTS_ID=row[1])
    # except Exception as e:
    #     print(e)
    #     print("CONTENTS_REC 삭제 에러")
    # conn.commit()

    cat_seq = 1
    print("TBH_CONTENTS_REC 저장중입니다.")
    for i in range(result_ori_count):
        cat_nam = result_ori2[i].split(':')[0]
        cat_pct_ai = result_ori2[i].split(':')[1]
        merge_ori_sql = "MERGE INTO TBH_CONTENTS_REC" \
                        " USING DUAL" \
                        " ON (CONTENTS_ID =:CONTENTS_ID AND REC_SEQ =:REC_SEQ)" \
                        " WHEN MATCHED THEN" \
                        " UPDATE SET REC_NAM=:REC_NAM , REC_COD=(SELECT CAT_SEQ FROM TBH_CONTENTS_REC_CAT" \  
                        " WHERE CAT_NAM =:REC_NAM AND ROWNUM = 1), REC_PCT_AI =:REC_PCT_AI, REC_PCT='', REG_DAT=SYSDATE" \
                        " WHEN NOT MATCHED THEN" \
                        " INSERT (CONTENTS_ID, REC_SEQ, REC_COD, REC_NAM, REC_PCT_AI, REC_GUBUN, REG_DAT)" \
                        " VALUES (:CONTENTS_ID, :REC_SEQ, (SELECT CAT_SEQ FROM TBH_CONTENTS_REC_CAT " \
                        " WHERE CAT_NAM =:REC_NAM AND ROWNUM = 1), :REC_NAM, :REC_PCT_AI, :REC_GUBUN, sysdate)"

        # insert_ori_sql = "INSERT INTO TBH_CONTENTS_REC (CONTENTS_ID, REC_SEQ, REC_COD, REC_NAM, REC_PCT_AI, REC_GUBUN, REG_DAT)" \
        #                  "VALUES (:CONTENTS_ID, :REC_SEQ, (SELECT CAT_SEQ FROM TBH_CONTENTS_REC_CAT WHERE CAT_NAM =:REC_NAM AND ROWNUM = 1), :REC_NAM, :REC_PCT_AI, :REC_GUBUN, sysdate)"
        cur.execute(merge_ori_sql, CONTENTS_ID=row[2], REC_SEQ=cat_seq, REC_NAM=cat_nam, REC_PCT_AI=cat_pct_ai,
                    REC_GUBUN='A')
        conn.commit()
        cat_seq = cat_seq + 1
    for i in range(result_rev_count):
        cat_nam = result_rev2[i].split(':')[0]
        cat_pct = result_rev2[i].split(':')[1]
        update_rev_sql = "UPDATE TBH_CONTENTS_REC " \
                         "   SET REC_PCT =:REC_PCT" \
                         " WHERE REC_NAM =:REC_NAM" \
                         "   AND CONTENTS_ID =:CONTENTS_ID"
        cur.execute(update_rev_sql, REC_PCT=cat_pct, REC_NAM=cat_nam, CONTENTS_ID=row[2])
        conn.commit()
    update_content(row, 'Y')
    print("TBH_CONTENTS_REC 저장완료")

In [ ]:
def save_board(rec_text, row):
    cur = conn.cursor()
    result_ori = rec_text.split('||')[0]
    result_rev = rec_text.split('||')[1]

    result_ori2 = result_ori.split(',')
    result_rev2 = result_rev.split(',')

    result_ori_count = len(result_ori2)
    result_rev_count = len(result_rev2)

    cat_seq = 1
    print("TBH_BOARD_CAT 저장중입니다.")
    for i in range(result_ori_count):
        cat_nam = result_ori2[i].split(':')[0]
        cat_pct_ai = result_ori2[i].split(':')[1]
        merge_ori_sql = "MERGE INTO TBH_BOARD_CAT" \
                        " USING DUAL" \
                        " ON (BOARD_KEY =:BOARD_KEY AND CAT_SEQ =:CAT_SEQ)" \
                        " WHEN MATCHED THEN" \
                        " UPDATE SET CAT_NAM=:CAT_NAM, CAT_COD = (SELECT CAT_SEQ FROM TBH_CONTENTS_REC_CAT " \
                        " WHERE CAT_NAM =:CAT_NAM AND ROWNUM = 1), CAT_PCT_AI =:CAT_PCT_AI, CAT_PCT='', REG_DAT=SYSDATE" \
                        " WHEN NOT MATCHED THEN" \
                        " INSERT (BOARD_KEY, CAT_SEQ, CAT_COD, CAT_NAM, CAT_PCT_AI, CAT_GUBUN, REG_DAT)" \
                        " VALUES (:BOARD_KEY, :CAT_SEQ, (SELECT CAT_SEQ FROM TBH_CONTENTS_REC_CAT " \
                        " WHERE CAT_NAM =:CAT_NAM AND ROWNUM = 1), :CAT_NAM, :CAT_PCT_AI, :CAT_GUBUN, sysdate)"

        cur.execute(merge_ori_sql, BOARD_KEY=row[2], CAT_SEQ=cat_seq, CAT_NAM=cat_nam, CAT_PCT_AI=cat_pct_ai,
                    CAT_GUBUN='A')
        conn.commit()
        cat_seq = cat_seq + 1
        if cat_seq == 21:
            break

    for i in range(result_rev_count):
        cat_nam = result_rev2[i].split(':')[0]
        cat_pct = result_rev2[i].split(':')[1]
        update_rev_sql = "UPDATE TBH_BOARD_CAT " \
                         "   SET CAT_PCT =:CAT_PCT" \
                         " WHERE CAT_NAM =:CAT_NAM" \
                         "   AND BOARD_KEY =:BOARD_KEY"
        cur.execute(update_rev_sql, CAT_PCT=cat_pct, CAT_NAM=cat_nam, BOARD_KEY=row[2])
        conn.commit()
    update_content(row, 'Y')
    print("TBH_BOARD_CAT 저장완료")

In [ ]:
def save_id_fav(rec_text, row):
    cur = conn.cursor()
    # print(id_cod)
    # print(rec_text)
    result_ori = rec_text.split('||')[0]
    result_rev = rec_text.split('||')[1]

    result_ori2 = result_ori.split(',')
    result_rev2 = result_rev.split(',')

    result_ori_count = len(result_ori2)
    result_rev_count = len(result_rev2)

    cat_seq = 1
    print("TBH_ID_CAT 저장중입니다.")
    for i in range(result_ori_count):
        cat_nam = result_ori2[i].split(':')[0]
        cat_pct_ai = result_ori2[i].split(':')[1]

        merge_ori_sql = "MERGE INTO TBH_ID_CAT" \
                        " USING DUAL" \
                        " ON (ID_COD =:ID_COD AND CAT_SEQ =:CAT_SEQ)" \
                        " WHEN MATCHED THEN" \
                        " UPDATE SET CAT_NAM=:CAT_NAM, CAT_PCT_AI =:CAT_PCT_AI, CAT_PCT='', REG_DAT=SYSDATE" \
                        " WHEN NOT MATCHED THEN" \
                        " INSERT (ID_COD, CAT_SEQ, CAT_COD, CAT_NAM, CAT_PCT_AI, CAT_GUBUN, REG_DAT)" \
                        " VALUES (:ID_COD, :CAT_SEQ, (SELECT CAT_SEQ FROM TBH_CONTENTS_REC_CAT " \
                        " WHERE CAT_NAM =:CAT_NAM AND ROWNUM = 1), :CAT_NAM, :CAT_PCT_AI, :CAT_GUBUN, sysdate)"

        cur.execute(merge_ori_sql, ID_COD=row[2], CAT_SEQ=cat_seq, CAT_NAM=cat_nam, CAT_PCT_AI=cat_pct_ai,
                    CAT_GUBUN='A')
        conn.commit()
        cat_seq = cat_seq + 1

    for i in range(result_rev_count):
        cat_nam = result_rev2[i].split(':')[0]
        cat_pct = result_rev2[i].split(':')[1]
        update_rev_sql = "UPDATE TBH_ID_CAT " \
                         "   SET CAT_PCT =:CAT_PCT" \
                         " WHERE CAT_NAM =:CAT_NAM" \
                         "   AND ID_COD =:ID_COD"
        cur.execute(update_rev_sql, CAT_PCT=cat_pct, CAT_NAM=cat_nam, ID_COD=row[2])
        conn.commit()
    update_content(row, 'Y')
    print("TBH_ID_CAT 저장완료")

In [ ]:
def update_content(row, status):
    cur = conn.cursor()
    if status == 'Y':
        sql = "UPDATE TBH_RECOMMEND_REQ" \
              " SET REQ_STATUS =:REQ_STATUS" \
              "   , END_DAT=SYSDATE" \
              " WHERE REQ_ID =:REQ_ID"
    try:
        cur.execute(sql, REQ_STATUS=status, REQ_ID=row[0])
        print(str(row[2]) + " " + status + " 업데이트 완료")
    except Exception as e:
        print(e)
        print("TBH_RECOMMEND_REQ " + status + " 업데이트 에러")
    conn.commit()

In [ ]:
def gettext_by_url(body_url):
    headers = {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    r = requests.get(body_url, headers=headers)
    r.encoding = 'UTF-8'
    soup = BeautifulSoup(r.text, 'html.parser')
    txt = soup.get_text()
    return txt

In [ ]:
if __name__ == "__main__":
    # p1 = Process(target=select_recommend_req)
    # p2 = Process(target=select_recommend_req)
    # p1.start()
    # p2.start()
    numbers = [5, 10, 15, 20, 25]
    procs = []

    for index, number in enumerate(numbers):
        proc = Process(target=select_recommend_req)
        procs.append(proc)
        proc.start()
    # select_recommend_req()
    for proc in procs:
        proc.join()
    conn.close()